In [14]:
import pandas as pd

# Read the CSV file with a specified encoding
df = pd.read_csv("cleaned_flight_data.csv", encoding="ISO-8859-1")

# Display the first 5 rows
df.head()

C:\Users\annik\AppData\Local\Temp\ipykernel_17160\1520107860.py:4: DtypeWarning: Columns (69,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cleaned_flight_data.csv", encoding="ISO-8859-1")


,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum
0,1998,1,1,2,5,1998-01-02,NW,19386,NW,N297US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009,2,5,28,4,2009-05-28,FL,20437,FL,N946AT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,2,6,29,6,2013-06-29,MQ,20398,MQ,N665MQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,3,8,31,2,2010-08-31,DL,19790,DL,N6705Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006,1,1,15,7,2006-01-15,US,20355,US,N504AU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Display the number of rows and columns in the dataset
print(f"\nShape of the dataset: {df.shape}")


Shape of the dataset: (2000000, 85)


**RandomForest Model**

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Assuming 'df' is already loaded

# Fill missing values only for numerical columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())

# Create target variable 'DelayCategory' for classification
def delay_category(row):
    if row['DepDelay'] > 0:
        return 'Departure Delay'
    elif row['ArrDelay'] > 0:
        return 'Arrival Delay'
    else:
        return 'No Delay'

df['DelayCategory'] = df.apply(delay_category, axis=1)

# Encode the categorical variable 'DelayCategory'
le = LabelEncoder()
df['DelayCategory'] = le.fit_transform(df['DelayCategory'])

# Ensure 'DepTime' and 'ArrTime' columns exist before creating hour/minute columns
if 'DepTime' in df.columns:
    df['DepTime'] = df['DepTime'].fillna(0).astype(int)
    df['DepTime_hour'] = df['DepTime'] // 100
    df['DepTime_minute'] = df['DepTime'] % 100
else:
    print("Column 'DepTime' is missing!")

if 'ArrTime' in df.columns:
    df['ArrTime'] = df['ArrTime'].fillna(0).astype(int)
    df['ArrTime_hour'] = df['ArrTime'] // 100
    df['ArrTime_minute'] = df['ArrTime'] % 100
else:
    print("Column 'ArrTime' is missing!")

# Feature engineering: Combine hour and minute into a single time feature
if 'DepTime_hour' in df.columns and 'DepTime_minute' in df.columns:
    df['DepTime_combined'] = df['DepTime_hour'] * 60 + df['DepTime_minute']
else:
    print("Skipping 'DepTime_combined' due to missing columns!")

if 'ArrTime_hour' in df.columns and 'ArrTime_minute' in df.columns:
    df['ArrTime_combined'] = df['ArrTime_hour'] * 60 + df['ArrTime_minute']
else:
    print("Skipping 'ArrTime_combined' due to missing columns!")

# Adding features based on the exploration
df['HighDelaySeason'] = df['Month'].apply(lambda x: 1 if x == 6 or x == 12 else 0)
df['LateNightFlight'] = df['DepTime_combined'].apply(lambda x: 1 if 180 <= x % 1440 <= 240 else 0)
df['LateArrivalFlight'] = df['ArrTime_combined'].apply(lambda x: 1 if 180 <= x % 1440 <= 240 else 0)

# New arrival-related features
df['CarrierDelay'] = df['DepDelay'].apply(lambda x: 1 if pd.notna(x) and x > 0 else 0)
df['WeatherDelay'] = df['ArrDelay'].apply(lambda x: 1 if pd.notna(x) and x > 0 else 0)
df['ArrDelayFlag'] = df['ArrDelay'].apply(lambda x: 1 if pd.notna(x) and x > 0 else 0)

# Features for training
features = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime_combined', 'ArrTime_combined',
            'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Flights', 'Distance', 
            'HighDelaySeason', 'LateNightFlight', 'LateArrivalFlight', 'ArrDelayFlag']

# Features matrix and target vector
X = df[features]
y = df['DelayCategory']

# Apply LabelEncoder to all categorical columns in X (if any)
label_encoder = LabelEncoder()
for col in X.select_dtypes(include=['object']).columns:
    X[col] = label_encoder.fit_transform(X[col].astype(str))

# Ensure all features are numeric and fill any remaining missing values
X = X.fillna(X.median())

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize RandomForestClassifier with optimized parameters
clf = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, class_weight='balanced')

# Train the model
clf.fit(X_train, y_train)

# Predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Accuracy Score:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.89      0.64     57077
           1       0.91      0.42      0.57    155057
           2       0.83      1.00      0.90    187866

    accuracy                           0.76    400000
   macro avg       0.74      0.77      0.71    400000
weighted avg       0.81      0.76      0.74    400000

Accuracy Score: 0.7577425
